<a href="https://colab.research.google.com/github/kyunghyuncho/ammi-2019-nlp/blob/master/01-day-LM/neural_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Language Modeling

In [1]:
import os
import sys
sys.path.append('utils/')
import loading_text_and_tokenization
import torch
import numpy as np
import torch.nn as nn
import random
import math

import utils.ngram_utils as ngram_utils
from utils.ngram_utils import NgramLM
from utils.amazon_dataset import AmazonDataset, pad, batchify
from torch.utils.data import DataLoader
from utils.neural_lm import BagOfNGrams, DecoderMLP, seq2seq
import utils.global_variables as gl
import torch
from tqdm import tqdm_notebook, tqdm
_tqdm = tqdm_notebook

In [2]:
torch.manual_seed(1)


In [3]:
use_cuda = True
device = torch.device("cuda" if (torch.cuda.is_available() and use_cuda) else "cpu")


In [4]:
# Read data from .txt files and create lists of reviews
train_data = []
# create a list of all the reviews 
with open('../data/new_train.txt', 'r') as f:
    train_data = [review for review in f.read().split('\n') if review]
    
valid_data = []
# create a list of all the reviews 
with open('../data/new_valid.txt', 'r') as f:
    valid_data = [review for review in f.read().split('\n') if review]
    

In [5]:
len(train_data), len(valid_data)

(22288, 2785)

In [6]:
train_data[0], valid_data[0]
train_data = train_data#[:100]
valid_data = valid_data#[:10]
train_data[0], type(train_data), len(train_data), type(train_data[0])

("this is a great tutu and at a really great price . it doesn ' t look cheap at all . i ' m so glad i looked on amazon and found such an affordable tutu that isn ' t made poorly . a + + ",
 list,
 22288,
 str)

In [7]:
# Tokenize the Datasets
# TODO: this takes a really long time !! why?
train_data_tokenized, all_tokens_train = ngram_utils.tokenize_dataset(train_data)
valid_data_tokenized, all_tokens_valid = ngram_utils.tokenize_dataset(valid_data)


In [8]:
train_data_tokenized[0], all_tokens_train[0]

(['this',
  'is',
  'a',
  'great',
  'tutu',
  'and',
  'at',
  'a',
  'really',
  'great',
  'price',
  '.'],
 'this')

In [9]:
N = 5

In [10]:
train_data_padded = ngram_utils.pad_dataset(train_data_tokenized, n=N)
valid_data_padded = ngram_utils.pad_dataset(valid_data_tokenized, n=N)

In [11]:
train_data_padded[0]

['<sos>',
 '<sos>',
 '<sos>',
 '<sos>',
 'this',
 'is',
 'a',
 'great',
 'tutu',
 'and',
 'at',
 'a',
 'really',
 'great',
 'price',
 '.',
 '<eos>',
 '<eos>',
 '<eos>',
 '<eos>']

In [12]:
vocab = ngram_utils.get_vocab(train_data_padded)
vocab_size = len(vocab)
vocab_size, vocab[:10]

(20805, ('<sos>', '<eos>', '.', 'the', 'i', ',', 'and', 'a', 'to', "'"))

In [13]:
id2token, token2id = ngram_utils.get_dict(vocab)
len(id2token), len(token2id)

(20809, 20807)

In [14]:
train_data_ids = ngram_utils.get_ids(train_data_padded, token2id)
valid_data_ids = ngram_utils.get_ids(valid_data_padded, token2id)

In [15]:
train_dataset = AmazonDataset(train_data_ids, max_inp_length=None, use_cuda=True)
train_dataset_ngrams = []
for t in train_dataset:
    for i in range(len(t) - N):
        train_dataset_ngrams.append((t[i:i + N], t[i + N]))
train_loader = DataLoader(train_dataset_ngrams, batch_size=2048, collate_fn=batchify, shuffle=True)

100%|██████████| 107790/107790 [00:07<00:00, 14038.55it/s]


In [16]:
valid_dataset = AmazonDataset(valid_data_ids, max_inp_length=None, use_cuda=True)
valid_dataset_ngrams = []
for t in valid_dataset:
    for i in range(len(t) - N):
        valid_dataset_ngrams.append((t[i:i + N], t[i + N]))
valid_loader = DataLoader(valid_dataset_ngrams, batch_size=2048, collate_fn=batchify, shuffle=True)


100%|██████████| 15172/15172 [00:00<00:00, 32566.68it/s]


In [17]:
train_loader

In [18]:
num_train = len(train_dataset_ngrams)
num_valid = len(valid_dataset_ngrams)
num_train, num_valid

(1946816, 280739)

In [28]:
encoder = BagOfNGrams(len(id2token), emb_dim=300, hidden_size=256, out_size=128, activation='ReLU', nlayers=2, reduce='mean', dropout=0.1, batch_norm=False)
encoder

BagOfNGrams(
  (embedding): EmbeddingBag(20809, 300, mode=mean)
  (layers): ModuleList(
    (0): Linear(in_features=300, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1)
    (3): Linear(in_features=256, out_features=128, bias=True)
  )
)

In [29]:
decoder = DecoderMLP(input_size=128, output_size=len(id2token), hidden_size=256)
decoder

DecoderMLP(
  (linear): Linear(in_features=128, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=20809, bias=True)
  (log_softmax): LogSoftmax()
)

In [30]:
model = seq2seq(encoder, decoder, id2token, use_cuda=False, lr=0.1, size_ngrams=N) 
model

seq2seq(
  (encoder): BagOfNGrams(
    (embedding): EmbeddingBag(20809, 300, mode=mean)
    (layers): ModuleList(
      (0): Linear(in_features=300, out_features=256, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1)
      (3): Linear(in_features=256, out_features=128, bias=True)
    )
  )
  (decoder): DecoderMLP(
    (linear): Linear(in_features=128, out_features=256, bias=True)
    (out): Linear(in_features=256, out_features=20809, bias=True)
    (log_softmax): LogSoftmax()
  )
  (criterion): NLLLoss()
)

## Training

In [31]:
num_epochs = 10
log_interval = 10
best_eval_loss = np.inf

for epoch in range(num_epochs):
    # Train
    train_loss = 0   
    cur_loss = 0
    for i, (data, labels) in _tqdm(enumerate(train_loader), disable=True):
        prediction, loss = model.train_step(data, labels)
        train_loss += len(data) * loss
        cur_loss += loss
        
        if i % log_interval == 0 and i > 0:
            cur_loss = cur_loss / log_interval
            print('| Epoch {:3d} | Train Loss {:5.2f} | Train PPL {:8.2f} | {:5d}/{:5d} Batches'.format(
                epoch, cur_loss, math.exp(cur_loss), i, int(num_train/len(data))))
            cur_loss = 0
    
    train_loss = train_loss / num_train
    print('| Epoch {:3d} | Train Loss {:5.2f} | Train PPL {:8.2f}'.format(
            epoch, train_loss, math.exp(train_loss)))

    # Eval
    if epoch % 1 == 0:        
        eval_loss = 0
        for i, (data, labels) in _tqdm(enumerate(valid_loader), disable=True):
            prediction, loss = model.train_step(data, labels, eval_mode=True)
            eval_loss += len(data) * loss
        eval_loss = eval_loss / num_valid 
        print('-' * 89)
        print('| Epoch {:3d} | Valid Loss {:5.2f} | Valid PPL {:8.2f}'.format(
            epoch, eval_loss, math.exp(eval_loss)))
        print('-' * 89)

        # Save the model if the validation loss is the best we've seen so far.
        if not best_eval_loss or eval_loss < best_eval_loss:
            with open('neural_lm_amazon_model' + '.pt', 'wb') as f:
                torch.save(model, f)
            best_eval_loss = eval_loss        


| Epoch   0 | Train Loss  9.80 | Train PPL 18103.76 |    10/  950 Batches
| Epoch   0 | Train Loss  8.08 | Train PPL  3217.01 |    20/  950 Batches
| Epoch   0 | Train Loss  7.31 | Train PPL  1495.15 |    30/  950 Batches
| Epoch   0 | Train Loss  6.27 | Train PPL   531.09 |    40/  950 Batches
| Epoch   0 | Train Loss  5.75 | Train PPL   312.67 |    50/  950 Batches
| Epoch   0 | Train Loss  5.48 | Train PPL   239.59 |    60/  950 Batches
| Epoch   0 | Train Loss  5.33 | Train PPL   205.62 |    70/  950 Batches
| Epoch   0 | Train Loss  5.28 | Train PPL   195.95 |    80/  950 Batches
| Epoch   0 | Train Loss  5.14 | Train PPL   171.37 |    90/  950 Batches
| Epoch   0 | Train Loss  5.11 | Train PPL   166.06 |   100/  950 Batches
| Epoch   0 | Train Loss  5.01 | Train PPL   149.35 |   110/  950 Batches
| Epoch   0 | Train Loss  5.07 | Train PPL   158.69 |   120/  950 Batches
| Epoch   0 | Train Loss  4.88 | Train PPL   131.24 |   130/  950 Batches
| Epoch   0 | Train Loss  4.96 | Train

Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f532f10ccf0>
Traceback (most recent call last):
  File "/home/roberta/miniconda3/envs/ammi/lib/python3.7/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


KeyboardInterrupt: 

In [34]:
def score_sentence(sent):
    tokenized, _ = ngram_utils.tokenize_dataset(sent)
    sent_ids = ngram_utils.get_ids(tokenized, token2id)
    sent_tensor = torch.LongTensor(sent_ids).to(device)
    generated, scores = model.eval_step(sent_tensor, score_only=True)
    ppl = math.exp(scores)
    return ppl

In [42]:
sent = ['this is a great tutu']
ppl = score_sentence(sent)
ppl

0.1470156052928668

In [43]:
sent = ['i do not like this tutu']
ppl = score_sentence(sent)
ppl

0.11621235433872693

In [44]:
sent = ['i will request a refund']
ppl = score_sentence(sent)
ppl

0.10470419285152689

In [46]:
sent = ['i do not understand']
ppl = score_sentence(sent)
ppl

0.09637728946146001

In [49]:
sent = ['this is a very cool watch .']
type(sent), len(sent), type(sent[0]), sent[0]
ppl = score_sentence(sent)
ppl

0.1783618638572311

In [83]:
sent = ['blah watch blah what']
type(sent), len(sent), type(sent[0]), sent[0]
ppl = score_sentence(sent)
ppl

0.07417229267866518

In [48]:
valid_data[:10]

["these are not sized right . a 3x is always big on me and these r cut wrong ! i ' m returning them . i think they are darling but they are just cut wrong . i got them in a timely fashion . there was no problem with the seller . just an issue with sizing . ",
 'a cool watch to have ',
 "i own several watches , and this is my favorite dress watch . had a breitling before , but it ' s high profile didn ' t work well with my dress shirts . the slim design of the bulova makes this clearly a classic dress watch . ",
 "i am happy with my recent purchase of the bulova men ' s 97b13 black tortoise shell leather strap watch . it was my first purchase of a bulova watch , and is amazed by the size and feel of my watch , ",
 'having read hundreds of reviews of watches on this website i chose this one to add to my collection : - likes : - an elegant and easy to set watch - utterly easy to use and simpledislikes - perhaps too small for anyone with big wrists , the wrist band does not feel like it is

In [57]:
def generate_sentence(context=None):
    if context is None:
        dummy_context = torch.LongTensor([[0]]).to(device)
        generated, scores = model.eval_step(dummy_context, use_context=False)
    else:
        tokenized, _ = ngram_utils.tokenize_dataset(context)
        context_ids = ngram_utils.get_ids(tokenized, token2id)
        context_tensor = torch.LongTensor(context_ids).to(device)
        generated, scores = model.eval_step(context_tensor, use_context=True)
    
    ppl = math.exp(scores)
    return generated, scores

In [93]:
generated, scores = generate_sentence()
print(' '.join(word[0] for word in generated))

the only reason i wish that they were made of a little the little the back . <eos>


In [98]:
generated, scores = generate_sentence(context=['this is not my'])
print(' '.join(word[0] for word in generated))

RuntimeError: invalid argument 3: out of range at /opt/conda/conda-bld/pytorch_1549633347309/work/aten/src/TH/generic/THTensor.cpp:350